In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from os import makedirs


In [ ]:
#plt.rcParams["font.family"] = "Times New Roman"
#plt.rcParams.update({'font.size': 18})

In [ ]:
# second way for font and size
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rcParams['font.size'] = 18

In [ ]:
file = 'sng_dataset_v1_valid2' #the name of a file.
try:
    makedirs(f"plots/{file}")
except FileExistsError:
    pass
sng_dataset_v1_unedited = pd.read_csv(f'{file}.csv')

In [ ]:
columns_to_delete = ["Unnamed: 0","MIDUPT1138P2300058", "SHORTUPT1138P2300058", "LONGUPT1138P2300058", "T1013P500399"]
sng_dataset_v1 = sng_dataset_v1_unedited.drop(columns = columns_to_delete)

In [ ]:
#SORTED BY WELL_ID AND TIME (FROM EARLIEST TO LATEST)
sng_dataset_v1 = sng_dataset_v1.sort_values(["WELL_ID", "UUID"])

Подготовка к построению графиков

In [ ]:
#DT variable change from string to DateTime
sng_dataset_v1['DT'] = pd.to_datetime(sng_dataset_v1['DT'], errors='coerce')

In [ ]:
#DataFrame with well_id and time when the first data was taken
pumps_numbers = sng_dataset_v1.drop_duplicates('WELL_ID')[['WELL_ID', 'DT']] # уникальные данные ID насосов
pumps_numbers = pumps_numbers.set_index('WELL_ID')
#pumps_numbers_for_file = sng_dataset_v1["WELL_ID"].drop_duplicates().tolist()


In [ ]:
for pump in pumps_numbers.index:
    try:
        makedirs(f"plots/{file}/{pump}")
    except FileExistsError:
        pass

In [ ]:
#how many days needed to be shown.
days_want_to_observe = 3
#locators
hours = mdates.HourLocator(interval = 12)
hour = mdates.HourLocator(interval = 1)

In [ ]:
def plotting_SETTINGS(pump, char, NAME, y_label):
  fig, ax = plt.subplots(figsize=(12,6))
  plt.plot(sng_dataset_v1[(sng_dataset_v1["WELL_ID"] == pump)]['DT'],
           sng_dataset_v1[(sng_dataset_v1["WELL_ID"] == pump)][char])
  ax.xaxis.set_major_locator(hours)
  ax.xaxis.set_minor_locator(hour)
  ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator(), show_offset= False,
                                                          formats = ['%H:%M \n %d %b, %Y', '%H:%M \n %d %b, %Y', '%H:%M \n %d %b, %Y', '%H:%M', '%H:%M', '%S.%f']))
  plt.xlim(pumps_numbers.loc[pump,'DT'], pumps_numbers.loc[pump,'DT']+timedelta(minutes=days_want_to_observe*24*60))
  plt.xlabel(r'$t$, час')
  plt.ylabel(y_label)
  plt.tight_layout()
  plt.savefig(f"plots/{file}/{pump}/v1_{pump}_{NAME}.jpg")
  plt.close()

In [ ]:
def making_plots(pump):

  # Наработка двигателя с момента последнего включения, сек ('T1138P6000096')
  plotting_SETTINGS(pump, 'T1138P6000096', "Наработка_двигателя_с_момента_последнего_включения_сек", r'$T_{r}$, сек')

  #Загрузка двигателя, % ('T1138P4000064')
  plotting_SETTINGS(pump, 'T1138P4000064', "Загрузка_двигателя_процент", r'$K_{l}$, %')

  #Коэффициент мощности (cos fi) ('T1138P600050')
  plotting_SETTINGS(pump, 'T1138P600050', "Коэффициент_мощности_cos_fi", 'cos(φ)')

  #Ток фазы A двигателя, А ('T1138P2600012')
  plotting_SETTINGS(pump, 'T1138P2600012', "Ток_фазы_A_двигателя_А", r'$T_{φ}$, А')

  #Средняя скорость изменения давления в коллекторе измерительной установки в часовом диапазоне усреднения, МПа/час ('DSHORTT1205P2300000')
  plotting_SETTINGS(pump, 'DSHORTT1205P2300000', "скорость_изменения_давления_в_коллекторе_мпа_час", r'$y$, МПа/час')

  #Средняя скорость изменения загрузки двигателя ЧАС, %/час ('DSHORTT1138P4000064')
  plotting_SETTINGS(pump, 'DSHORTT1138P4000064', "скорость_изменения_загрузки_процент_час", r'$ΔK_{l}/Δ𝑡$, %/час')

for p in pumps_numbers.index:
    making_plots(p)

In [ ]:
#automatic archive
import shutil
shutil.make_archive('plots', 'zip','plots')

In [ ]:
#download files
from google.colab import files
files.download("/content/plots.zip")


'\nfrom google.colab import files\nfiles.download("/content/plots.zip")\n'

In [ ]:
"""pump = 226002148
hours = mdates.HourLocator(interval = 2)
hour = mdates.HourLocator(interval = 1)
fig, ax = plt.subplots(figsize=(25,6))
plt.plot(sng_dataset_v1[(sng_dataset_v1["WELL_ID"] == pump)]['DT'],
         sng_dataset_v1[(sng_dataset_v1["WELL_ID"] == pump)]['T1138P6000096'])
ax.xaxis.set_major_locator(hours)
ax.xaxis.set_minor_locator(hour)
ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator(),
                                                         show_offset= False,
                                                         formats = ['%H:%M \n %d %b, %Y', '%H:%M \n %d %b, %Y', '%H:%M \n %d %b, %Y', '%H:%M', '%H:%M', '%S.%f']))
plt.xlim(sng_dataset_v1["DT"].min(), sng_dataset_v1["DT"].min()+timedelta(minutes=3*24*60))
fig.autofmt_xdate()
sng_dataset_v1["DT"].min()"""

'pump = 226002148\nhours = mdates.HourLocator(interval = 2)\nhour = mdates.HourLocator(interval = 1)\nfig, ax = plt.subplots(figsize=(25,6))\nplt.plot(sng_dataset_v1[(sng_dataset_v1["WELL_ID"] == pump)][\'DT\'], \n         sng_dataset_v1[(sng_dataset_v1["WELL_ID"] == pump)][\'T1138P6000096\'])\nax.xaxis.set_major_locator(hours)\nax.xaxis.set_minor_locator(hour)\nax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator(), \n                                                         show_offset= False,\n                                                         formats = [\'%H:%M \n %d %b, %Y\', \'%H:%M \n %d %b, %Y\', \'%H:%M \n %d %b, %Y\', \'%H:%M\', \'%H:%M\', \'%S.%f\']))\nplt.xlim(sng_dataset_v1["DT"].min(), sng_dataset_v1["DT"].min()+timedelta(minutes=3*24*60))\nfig.autofmt_xdate()\nsng_dataset_v1["DT"].min()'

In [ ]:
"""data = sng_dataset_v1[(sng_dataset_v1["WELL_ID"] == 226000188)]['T1138P6000096']
fig, ax = plt.subplots(figsize=(25,6))

df = pd.DataFrame(sng_dataset_v1[(sng_dataset_v1["WELL_ID"] == 226000188)]['T1138P6000096'])
df = df.set_index(data_time)


plt.plot(df.index, df.values)

ax.xaxis.set_major_locator(mdates.DayLocator(bymonthday=(1,3)))
ax.xaxis.set_minor_locator(mdates.HourLocator(byhour=(1)))

ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y %H:%M:%S'))
ax.xaxis.set_minor_formatter(mdates.DateFormatter('%H:%M:%S'))"""

'data = sng_dataset_v1[(sng_dataset_v1["WELL_ID"] == 226000188)][\'T1138P6000096\']\nfig, ax = plt.subplots(figsize=(25,6))\n\ndf = pd.DataFrame(sng_dataset_v1[(sng_dataset_v1["WELL_ID"] == 226000188)][\'T1138P6000096\'])\ndf = df.set_index(data_time)\n\n\nplt.plot(df.index, df.values)\n\nax.xaxis.set_major_locator(mdates.DayLocator(bymonthday=(1,3)))\nax.xaxis.set_minor_locator(mdates.HourLocator(byhour=(1)))\n\nax.xaxis.set_major_formatter(mdates.DateFormatter(\'%d-%m-%Y %H:%M:%S\'))\nax.xaxis.set_minor_formatter(mdates.DateFormatter(\'%H:%M:%S\'))'